<a href="https://colab.research.google.com/github/Shopping-Yuan/ML_Project/blob/Shopping_vscode_branch/Self_Attention_Classification/conformer_classification_voice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import shutil
shutil.copyfile('/content/drive/MyDrive/Dataset.zip','/content/Dataset.zip')

import zipfile
with zipfile.ZipFile('/content/Dataset.zip','r') as zip_ref:
  zip_ref.extractall('/content/')

In [3]:
def parse_args():
  """arguments"""
  config = {
#    "data_dir": "/content/Dataset",
    "data_dir": "./Dataset",
    "save_path": "model.ckpt",
    "batch_size": 32,
    "n_workers": 0,
    "valid_steps": 2000,
    "warmup_steps": 1000,
    "save_steps": 4000,
    "total_steps": 120000,
  }

  return config

#Dataset

In [4]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence


class myDataset(Dataset):
  def __init__(self, data_dir, segment_len=128):
    self.data_dir = data_dir
    self.segment_len = segment_len

    # Load the mapping from speaker neme to their corresponding id.
    # Path package can use / to join path
    #part 1-1
    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())
    self.speaker2id = mapping["speaker2id"]
    #self.speaker2id = {'id10001': 0, 'id10005': 1, ...... 'id11247': 598, 'id11251': 599}

    #part 1-2
    # Load metadata of training data.
    metadata_path = Path(data_dir) / "metadata.json"
    metadata = json.load(open(metadata_path))["speakers"]

    # Get the total number of speaker.
    self.speaker_num = len(metadata.keys())
    self.data = []
    for speaker in metadata.keys():
    #['id10473', 'id10328', 'id10382',......'id10864']
      for utterances in metadata[speaker]:
      # for example : metadata[speaker] = 'id10473'
      # utterances = {'feature_path': 'uttr-5c88b2f1803449789c36f14fb4d3c1eb.pt', 'mel_len': 652}......
        self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
        # for example : ['uttr-5c88b2f1803449789c36f14fb4d3c1eb.pt',217]

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    feat_path, speaker = self.data[index]
    #[utterances["feature_path"], self.speaker2id[speaker]

    # Load preprocessed mel-spectrogram.
    # https://pytorch.org/docs/stable/generated/torch.load.html
    # using torch.load to open pt file . By default , tensors are loaded to GPU if they are GPU tensor.
    # part 2
    mel = torch.load(os.path.join(self.data_dir, feat_path))

    # Segmemt mel-spectrogram into "segment_len" frames.
    if len(mel) > self.segment_len:
      # Randomly get the starting point of the segment.
      start = random.randint(0, len(mel) - self.segment_len)
      # Get a segment with "segment_len" frames.
      mel = torch.FloatTensor(mel[start:start+self.segment_len])
    else:
      mel = torch.FloatTensor(mel)
    # Turn the speaker id into long for computing loss later.
    speaker = torch.FloatTensor([speaker]).long()
    return mel, speaker

  def get_speaker_number(self):
    return self.speaker_num

In [5]:
#data_dir = parse_args()["data_dir"]
#part 1-1
#mapping.json checcking
#mapping_path = Path(data_dir) / "mapping.json"
#mapping = json.load(mapping_path.open())
#speaker2id = mapping["speaker2id"]
#print(type(mapping)) #<class 'dict'>
#print(mapping.keys()) #dict_keys(['speaker2id', 'id2speaker'])
#print(mapping.values()) #dict_values([{'id10001': 0, 'id10005': 1, ...... 'id11247': 598, 'id11251': 599},
             #{"0": "id10001", ......'599':  'id11251'}])
#print(mapping['speaker2id']['id10473']) #217

#part 1-2
#metadata.json checcking
#metadata_path = Path(data_dir) / "metadata.json"
#metadata = json.load(open(metadata_path))["speakers"]
#print(type(metadata)) #<class 'dict'>
#print(len(metadata.keys()),metadata.keys()) #600 dict_keys(['id10473', 'id10328', 'id10382',......'id10864'])
#print(len(metadata.values()),len(list(metadata.values())[0]),list(metadata.values())[0])
#600 176 [{'feature_path': 'uttr-5c88b2f1803449789c36f14fb4d3c1eb.pt', 'mel_len': 652},......
#{'feature_path': 'uttr-caf385f0032140e0add0bbde9a6595cd.pt', 'mel_len': 414}]
#print(len(metadata.values()),len(list(metadata.values())[1])) #600 101

#part 2
#mel = torch.load(os.path.join(data_dir, 'uttr-5c88b2f1803449789c36f14fb4d3c1eb.pt'))
#print(type(mel)) #<class 'torch.Tensor'>
#print(mel.size()) #torch.Size([652, 40])
#print(mel[0])
# tensor([
# 2.0060,  1.9015,  1.3473,  1.5711,  1.5569,  2.3200,  3.0064,  3.3848,
# 0.5376,  0.8397,  1.5710, -0.5662, -2.1262, -1.1271, -1.4272, -3.6233,
# -1.0765, -1.8453, -3.2415, -2.3427, -1.5888, -0.4692, -0.4291, -0.4722,
# -1.8450, -1.0638, -3.2177, -3.4006, -3.2176, -3.3409, -3.1142, -4.0395,
# -3.6502, -4.2407, -4.9656, -5.1238, -5.0620, -5.4308, -6.0757, -6.6186])

In [6]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
#part 1 check collate_batch
def collate_batch(batch):
  # Process features within a batch.
  """Collate a batch of data."""
  #part 1-1 , 1-2
  mel, speaker = zip(*batch)
  # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
  mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
  # mel: (batch size, length, 40)
  #part 1-3
  return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
  """Generate dataloader"""
  dataset = myDataset(data_dir)
  speaker_num = dataset.get_speaker_number()
  # Split dataset into training dataset and validation dataset
  trainlen = int(0.9 * len(dataset))
  lengths = [trainlen, len(dataset) - trainlen]
  trainset, validset = random_split(dataset, lengths)

  train_loader = DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=collate_batch,
  )
  valid_loader = DataLoader(
    validset,
    batch_size=batch_size,
    num_workers=n_workers,
    drop_last=True,
    pin_memory=True,
    collate_fn=collate_batch,
  )

  return train_loader, valid_loader, speaker_num

In [7]:
# x = [["a",1],["b",2],["c",3]]
# print(list(zip(*x)))
# print(list(zip(("a",1),("b",2),("c",3))))

In [8]:
#import numpy as np
#check collate_batch
#part 1-1
# * collect "sequence"function input to a tuple :
#def f(*paras):
#  print(paras)
#f(1,2,3) #(1, 2, 3)
# or collect a tuple to a "sequence"
#print(*(1,2,3)) #1 2 3

#part 1-2
#test_data = np.random.rand(2,3,4)
#test_label = np.random.rand(2,1)
#samples = (test_data,test_label)
#print(samples)
#x , y = zip((1,2),(3,4),(5,6),(7,8))
#print(x,y)

#part 1-3
#print(torch.FloatTensor([[1.],[1.]]))#tensor([[1.],[1.]])
#print(torch.FloatTensor([[1.],[1.]]).long())#tensor([[1],[1]]) implies the numbers are integer now
#print(torch.IntTensor([[1.],[1.]]))#tensor([[1],[1]], dtype=torch.int32)

In [9]:
!pip install torchinfo
#%conda install -c conda-forge torchinfo

In [11]:
import torch.nn as nn
class PrintLayer(nn.Module):
  def __init__(self):
    super().__init__()
  def forward(self , x):
    print(f"layer output size = {x.size()}")
    return(x)
class ConformerLayer(nn.Module):
  """
    #------Conformer Encoder Part----------------------------------
    # part 1 feed forward module
    # part 2 multi heads self attention
    # part 3 convolution module
    # part 4 feed forward module
    # part 5 layer norm
    # multi heads self attention module
    #   part 2-1 laryer norm
    #   part 2-2 multi heads self attention
    #   part 2-3 dropout
    # convolution module
    #   part 3-1 layer norm (do this outside the module)
    #   part 3-2 pointwise conv
    #   part 3-3 Glu activation
    #   part 3-4 1D depthwise conv
    #   part 3-5 batch norm
    #   part 3-6 swish activation
    #   part 3-7 pointwise conv
    #   part 3-8 dropout
    """
  def __init__(self,d_model,feedforword,dropout):
    super().__init__()
    self.feed_forward_layer_1 = nn.Sequential(
 #   nn.BatchNorm1d(d_model),
    nn.Linear(d_model, feedforword),
    nn.ReLU(),
    nn.Linear(feedforword, d_model),
 #   nn.BatchNorm1d(d_model),
    )
    self.layer_norm1 = nn.LayerNorm(d_model)
    self.layer_norm2 = nn.LayerNorm(d_model)
    self.attention = torch.nn.MultiheadAttention(d_model, num_heads = 2, dropout=dropout)
    self.DropoutMA = nn.Dropout(dropout)
    self.cnn_module = nn.Sequential(
      # part 3-2
      nn.Conv1d(in_channels = d_model , out_channels = d_model*2 , kernel_size = 1),
      # part 3-3
      nn.GLU(dim = 1),
      # part 3-4
      nn.Conv1d(in_channels = d_model , out_channels = d_model , kernel_size = 3 ,padding = 1, groups = d_model),
      # part 3-5
      nn.BatchNorm1d(d_model),
      # part 3-6
      nn.SiLU(),
      # part 3-7
      nn.Conv1d(in_channels = d_model , out_channels = d_model , kernel_size = 1),
      # part 3-8
      nn.Dropout1d()
    )
    self.feed_forward_layer_2 = nn.Sequential(
 #   nn.BatchNorm1d(d_model),
    nn.Linear(d_model, feedforword),
    nn.ReLU(),
    nn.Linear(feedforword, d_model),
 #   nn.BatchNorm1d(d_model),
    )
    self.layer_norm3 = nn.LayerNorm(d_model)
  def forward(self ,x ):
    # part 1
    x = x + 0.5 * self.feed_forward_layer_1(x)
    # part 2
    x = x.permute(1, 0, 2)
    x = self.layer_norm1(x)
    x = x + self.attention(x,x,x, need_weights=False)[0]
    x = self.DropoutMA(x)
    # part 3
    x = self.layer_norm2(x)
    x = x.permute(1, 2, 0)
    x = x + self.cnn_module(x)
    x = x.permute(0, 2, 1)

    # part 4
    x = x + 0.5 * self.feed_forward_layer_2(x)
    # part 5
    x = self.layer_norm3(x)
    return x
class SelfAttentionPollingLayer(nn.Module):
  def __init__(self,layer):
    super().__init__()
    self.layer = layer
  def forward(self ,x ):

    return torch.squeeze(torch.bmm(x.permute(0,2,1),x),1)

In [12]:
# x = torch.rand(5,2,3)
# x = nn.Dropout1d()(x)
# print(x)

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchinfo import summary

class Classifier(nn.Module):
  def __init__(self, d_model=128, n_spks=600, feedforward = 256 , dropout=0.2):
    super().__init__()
    # Project the dimension of features from that of input into d_model.
    self.prenet = nn.Linear(40, d_model)
    # TODO:

    # attension pooling
    self.attension_pooling = nn.Sequential(
      nn.Linear(d_model, d_model),
      nn.ReLU(),
      nn.Linear(d_model, 1),
      nn.BatchNorm1d(d_model),
      nn.Softmax(dim = 1),
    )
#    self.conformer = ConformerLayer(d_model,feedforward,dropout)
    self.conformers = nn.ModuleList([ConformerLayer(d_model,feedforward,dropout) for i in range(2)])
    self.pred_layer = nn.Sequential(
      nn.Linear(d_model, n_spks),
    )

  def forward(self, mels):
    """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """

    out = self.prenet(mels)
    for layer in self.conformers:
      out = layer(out)


    sfatpool = self.attension_pooling(out)
    out = torch.squeeze(torch.bmm(sfatpool.permute(0,2,1),out),1)
    out = self.pred_layer(out)
    return out


In [14]:
model = Classifier(d_model=128, dropout=0.2)
#model = ConformerLayer(d_model=80, feedforword = 256, dropout=0.2)
print(summary(model,(32,128,40)))

Layer (type:depth-idx)                   Output Shape              Param #
Classifier                               [32, 600]                 --
├─Linear: 1-1                            [32, 128, 128]            5,248
├─ModuleList: 1-2                        --                        --
│    └─ConformerLayer: 2-1               [32, 128, 128]            --
│    │    └─Sequential: 3-1              [32, 128, 128]            65,920
│    │    └─LayerNorm: 3-2               [128, 32, 128]            256
│    │    └─MultiheadAttention: 3-3      [128, 32, 128]            66,048
│    │    └─Dropout: 3-4                 [128, 32, 128]            --
│    │    └─LayerNorm: 3-5               [128, 32, 128]            256
│    │    └─Sequential: 3-6              [32, 128, 128]            50,304
│    │    └─Sequential: 3-7              [32, 128, 128]            65,920
│    │    └─LayerNorm: 3-8               [32, 128, 128]            256
│    └─ConformerLayer: 2-2               [32, 128, 128]        

In [15]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torchvision
# from torchinfo import summary

# class Classifier(nn.Module):
#   def __init__(self, d_model=128, n_spks=600, dropout=0.1):
#     super().__init__()
#     # Project the dimension of features from that of input into d_model.
#     self.prenet = nn.Linear(40, d_model)
#     # TODO:
#     #   Change Transformer to Conformer.
#     #   https://arxiv.org/abs/2005.08100
#     # embed_dim must be divisible by num_heads
#     self.encoder_layer = nn.TransformerEncoderLayer(
#       d_model=d_model, dim_feedforward=256, nhead=1, norm_first = True
#     )
#     # batch_first = True
#     # norm_first = True
#     self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers= 5  )
#     # attension pooling
#     self.attension_pooling = nn.Sequential(
#       nn.Linear(d_model, d_model),
#       nn.ReLU(),
#       nn.Linear(d_model, 1),
#       nn.Softmax(dim = 1),
#     )
# #    self.afd = SelfAttentionPollingLayer(self.attension_pooling)
# #    self.linear = nn.Linear(d_model, d_model)
# #    self.relu = nn.ReLU()
# #    self.linear_out = nn.Linear(d_model, n_spks)

#     # Project the the dimension of features from d_model into speaker nums.
#     self.pred_layer = nn.Sequential(
# #      nn.Linear(d_model, d_model),
# #      nn.ReLU(),
#       nn.Linear(d_model, n_spks),
#     )

#   def forward(self, mels):
#     """
#     args:
#       mels: (batch size, length, 40)
#     return:
#       out: (batch size, n_spks)
#     """
#     # out: (batch size, length, d_model)
#     #print(mels.size())
#     out = self.prenet(mels)
#     # out: (length, batch size, d_model)
#     out = out.permute(1, 0, 2)
#     #print(out.size())
#     # The encoder layer expect features in the shape of (length, batch size, d_model).
#     #out = self.encoder_layer(out)
#     out = self.encoder_layer(out)
#     # out: (batch size, length, d_model)
#     #print(out.size())
#     out = out.transpose(0, 1)
# #    print(out.size())
#     # self-attention pooling
#     #out = self.afd(out)
# #    print(out.size())
#     sfatpool = self.attension_pooling(out)
# #    print(sfatpool.size(),out.size())
#     out = torch.squeeze(torch.bmm(sfatpool.permute(0,2,1),out),1)
# #    out = self.afd(out)
# #    print(out.size())

#     # mean pooling
# #    stats = out.mean(dim=1)

#     # out: (batch, n_spks)
# #    out = self.pred_layer(stats)
# #    out = self.linear(out)
# #    out = self.relu(out)
# #    out = self.linear_out(out)
#     out = self.pred_layer(out)
#     return out


In [16]:
model = Classifier()
print(summary(model,(32,128,40)))
#print(model.state_dict().keys())
#print(model.state_dict()["encoder_layer.self_attn.in_proj_weight"].size())#torch.Size([240, 80]) (80+80+80)*80
#print(model.state_dict()["encoder_layer.self_attn.out_proj.weight"].size())#torch.Size([80, 80])

Layer (type:depth-idx)                   Output Shape              Param #
Classifier                               [32, 600]                 --
├─Linear: 1-1                            [32, 128, 128]            5,248
├─ModuleList: 1-2                        --                        --
│    └─ConformerLayer: 2-1               [32, 128, 128]            --
│    │    └─Sequential: 3-1              [32, 128, 128]            65,920
│    │    └─LayerNorm: 3-2               [128, 32, 128]            256
│    │    └─MultiheadAttention: 3-3      [128, 32, 128]            66,048
│    │    └─Dropout: 3-4                 [128, 32, 128]            --
│    │    └─LayerNorm: 3-5               [128, 32, 128]            256
│    │    └─Sequential: 3-6              [32, 128, 128]            50,304
│    │    └─Sequential: 3-7              [32, 128, 128]            65,920
│    │    └─LayerNorm: 3-8               [32, 128, 128]            256
│    └─ConformerLayer: 2-2               [32, 128, 128]        

original
parameter number :
1.MultiheadAttention : =
d_model+1(bias) * d_query +
d_model+1(bias) * d_key  +
d_model+1(bias) * d_v   +
d_v + 1(bias) * output_dim( = dmodel)
= (80+1)80 + (80+1)80 + (80+1)80 + (80+1)80 = 25920
2.layer_norm : =input + input = 80*80 = 160

first change
parameter number :
1.MultiheadAttention : =
d_model+1(bias) * d_query +
d_model+1(bias) * d_key  +
d_model+1(bias) * d_v   +
d_v + 1(bias) * output_dim( = dmodel)
= (128+1)128(3+1)= 16512

second change
parameter number :
1.MultiheadAttention : =
d_model+1(bias) * d_query +
d_model+1(bias) * d_key  +
d_model+1(bias) * d_v   +
d_v + 1(bias) * output_dim( = dmodel)
= (128+1)128(3+1)= 16512
four layers : 66048

2.layer_norm : =input + input = 80*80 = 160

3rd change
parameter number :
1.MultiheadAttention : =
d_model+1(bias) * d_query +
d_model+1(bias) * d_key  +
d_model+1(bias) * d_v   +
d_v + 1(bias) * output_dim( = dmodel)
= (128+1)128(3+1)= 16512
four layers : 66048

2.self-attension polling : (128+1)128+(128+1)1=16512+129=16641

4th change
parameter number :
1.MultiheadAttention : =
d_model+1(bias) * d_query +
d_model+1(bias) * d_key  +
d_model+1(bias) * d_v   +
d_v + 1(bias) * output_dim( = dmodel)
= (128+1)128(3+1)= 16512

2 layers

2.self-attension polling : (128+1)128+(128+1)1=16512+129=16641

3.delete on Linear + ReLU layers = - (129*128) -0 = -16512

4.batchnormalization

5th change
parameter number :
1.MultiheadAttention : =
d_model+1(bias) * d_query +
d_model+1(bias) * d_key  +
d_model+1(bias) * d_v   +
d_v + 1(bias) * output_dim( = dmodel)
= (128+1)128(3+1)= 16512

2 layers

2.self-attension polling : (128+1)128+(128+1)1=16512+129=16641

3.delete on Linear + ReLU layers = - (129*128) -0 = -16512

In [17]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
  optimizer: Optimizer,
  num_warmup_steps: int,
  num_training_steps: int,
  num_cycles: float = 0.5,
  last_epoch: int = -1,
):
  """
  Create a schedule with a learning rate that decreases following the values of the cosine function between the
  initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
  initial lr set in the optimizer.

  Args:
    optimizer (:class:`~torch.optim.Optimizer`):
      The optimizer for which to schedule the learning rate.
    num_warmup_steps (:obj:`int`):
      The number of steps for the warmup phase.
    num_training_steps (:obj:`int`):
      The total number of training steps.
    num_cycles (:obj:`float`, `optional`, defaults to 0.5):
      The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
      following a half-cosine).
    last_epoch (:obj:`int`, `optional`, defaults to -1):
      The index of the last epoch when resuming training.

  Return:
    :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
  """

  def lr_lambda(current_step):
    # Warmup
    if current_step < num_warmup_steps:
      return float(current_step) / float(max(1, num_warmup_steps))
    # decadence
    # cosine angle
    progress = float(current_step - num_warmup_steps) / float(
      max(1, num_training_steps - num_warmup_steps)
    )
    return max(
      0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
    )

  return LambdaLR(optimizer, lr_lambda, last_epoch)

In [18]:
#from torch.optim import AdamW
#optimizer = AdamW(model.parameters(), lr=1e-3)

#scheduler = get_cosine_schedule_with_warmup(optimizer, 1000, 70000)

#print(type(scheduler.optimizer.param_groups)) #<class 'list'>
#print(len(scheduler.optimizer.param_groups))  #1
#print(type(scheduler.optimizer.param_groups[0])) #<class 'dict'>
#print(scheduler.optimizer.param_groups[0].keys())
#['params', 'lr', 'betas', 'eps', 'weight_decay', 'amsgrad', 'foreach',
#'maximize', 'capturable', 'differentiable', 'fused', 'initial_lr']
#print(scheduler.optimizer.param_groups[0]['params'])
#[Parameter containing: tensor([[......],requires_grad=True)]
#print(scheduler.optimizer.param_groups[0]['lr'])# 0.0
#print(scheduler.optimizer.param_groups[0]['betas'])# (0.9, 0.999)
#print(scheduler.optimizer.param_groups[0]['eps']) # 1e-08
#print(scheduler.optimizer.param_groups[0]['weight_decay']) #0.01
#print(scheduler.optimizer.param_groups[0]['amsgrad']) # False
#print(scheduler.optimizer.param_groups[0]['foreach']) # None

In [19]:
import torch


def model_fn(batch, model, criterion, device):
  """Forward a batch through the model."""

  mels, labels = batch
  mels = mels.to(device)
  labels = labels.to(device)

  outs = model(mels)

  loss = criterion(outs, labels)

  # Get the speaker id with highest probability.
  preds = outs.argmax(1)
  # Compute accuracy.
  accuracy = torch.mean((preds == labels).float())

  return loss, accuracy

In [20]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device):
  """Validate on validation set."""

  model.eval()
  running_loss = 0.0
  running_accuracy = 0.0
  # Manually update the progress bar : step 1
  # total length
  pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")
  for i, batch in enumerate(dataloader):
    with torch.no_grad():
      loss, accuracy = model_fn(batch, model, criterion, device)
      running_loss += loss.item()
      running_accuracy += accuracy.item()
    # Manually update the progress bar : step 2
    # update each batch
    pbar.update(dataloader.batch_size)
    pbar.set_postfix(
      loss=f"{running_loss / (i+1):.2f}",
      accuracy=f"{running_accuracy / (i+1):.2f}",
    )
  # Manually update the progress bar : step 3
  pbar.close()
  model.train()

  return running_accuracy / len(dataloader)


In [ ]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split
import numpy as np

# def parse_args():
#   """arguments"""
#   config = {
#     "data_dir": "/content/Dataset",
#     "save_path": "model.ckpt",
#     "batch_size": 32,
#     "n_workers": 8,
#     "valid_steps": 2000,
#     "warmup_steps": 1000,
#     "save_steps": 10000,
#     "total_steps": 70000,
#   }

#   return config


def main(
  data_dir,
  save_path,
  batch_size,
  n_workers,
  valid_steps,
  warmup_steps,
  total_steps,
  save_steps,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
  train_iterator = iter(train_loader)
  print(f"[Info]: Finish loading data!",flush = True)
  #
  torch.backends.cudnn.benchmark = True
  torch.backends.cudnn.deterministic = True
  myseed = 1
  np.random.seed(myseed)
  torch.manual_seed(myseed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

  model = Classifier(n_spks=speaker_num).to(device)
  #model.load_state_dict(torch.load(save_path))
  criterion = nn.CrossEntropyLoss()
  optimizer = AdamW(model.parameters(), lr=1e-3)
  scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
  print(f"[Info]: Finish creating model!",flush = True)

  best_accuracy = -1.0
  best_state_dict = None
  # pbar has total 2000 step
  pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")
  # print train loss and accuracy every 1000 step

  loss_acc_list = []

  for step in range(total_steps):
    # Get data
    try:
      batch = next(train_iterator)
    except StopIteration:
      train_iterator = iter(train_loader)
      batch = next(train_iterator)

    loss, accuracy = model_fn(batch, model, criterion, device)
    batch_loss = loss.item()
    batch_accuracy = accuracy.item()

    # Updata model
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    # print train loss acc
    loss_acc_list.append((loss, accuracy))
    print_loss , print_acc = list(zip(*loss_acc_list))
    if step % 1000 == 0:
      print(f"average loss = {sum(print_loss)/len(print_loss):.4f} \
      average accuracy = {sum(print_acc)/len(print_acc):.4f})")
      loss_acc_list = []

    # Log
    # update each step (as total)
    pbar.update()
    pbar.set_postfix(
      loss=f"{batch_loss:.2f}",
      accuracy=f"{batch_accuracy:.2f}",
      step=step + 1,
    )

    # Do validation
    # valid each 2000 step
    if (step + 1) % valid_steps == 0:
      #close above pbar
      pbar.close()

      valid_accuracy = valid(valid_loader, model, criterion, device)

      # keep the best model
      if valid_accuracy > best_accuracy:
        best_accuracy = valid_accuracy
        best_state_dict = model.state_dict()
      # new pbar has total 2000 step
      pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    # Save the best model so far.
    if (step + 1) % save_steps == 0 and best_state_dict is not None:
      torch.save(best_state_dict, save_path)
      pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

  pbar.close()


if __name__ == "__main__":
  main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train:   0% 2/2000 [00:02<29:57,  1.11 step/s, accuracy=0.00, loss=6.46, step=2]

average loss = 6.4762       average accuracy = 0.0000)


Train:  50% 1001/2000 [01:39<01:42,  9.71 step/s, accuracy=0.16, loss=4.30, step=1001]

average loss = 5.5150       average accuracy = 0.0546)


...

Train:  50% 1001/2000 [01:27<01:15, 13.16 step/s, accuracy=0.97, loss=0.24, step=47001]

average loss = 0.6632       average accuracy = 0.8245)


Train: 100% 2000/2000 [02:42<00:00, 12.27 step/s, accuracy=0.81, loss=0.60, step=48000]
Valid: 100% 6944/6944 [00:17<00:00, 407.17 uttr/s, accuracy=0.77, loss=0.97]
Train:   0% 1/2000 [00:00<05:14,  6.36 step/s, accuracy=0.81, loss=0.52, step=48001]

Step 48000, best model saved. (accuracy=0.7800)
average loss = 0.6155       average accuracy = 0.8367)


Train:  12% 232/2000 [00:22<02:50, 10.36 step/s, accuracy=0.88, loss=0.68, step=48232]

##### original : val acc = 68%
##### d_model 128 , multihead 1
##### train acc = 80% , val acc = 73%

##### TransformerEncoder layer : 1 -> 4
##### Step : 70000 -> 100000
##### train acc = 92% , val acc = 83%

##### TransformerEncoder layer : 1 -> 4
##### stat pooling -> self attention pooling
##### Step : 100000 -> 120000
##### train acc = 95% , val acc = 86%

##### TransformerEncoder layer : 1 -> 5
##### stat pooling -> self attention pooling
##### prep layer : only keep last layer
##### Step : 70000 -> 56000(imcomplete)
##### train acc = 90% , val acc = 83%(imcomplete)

##### ConformerEncoder layer : 2
##### multihead 1 -> 2
##### add batch normalization in feedforward layers
##### stat pooling -> self attention pooling
##### prep layer : only keep last layer
##### train acc = 93% , val acc = 84%

##### ConformerEncoder layer : 2
##### multihead 1 -> 2
##### stat pooling -> self attention pooling
##### train acc = 93% , val acc = 86%

In [ ]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

class InferenceDataset(Dataset):
  def __init__(self, data_dir,segment_len = 128):
    # using / to merge path
    testdata_path = Path(data_dir) / "testdata.json"
    metadata = json.load(testdata_path.open())
    self.data_dir = data_dir
    #
    self.data = metadata["utterances"]
    self.segment_len = segment_len


  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    utterance = self.data[index]
    feat_path = utterance["feature_path"]

    mel = torch.load(os.path.join(self.data_dir, feat_path))
    mel = pad_sequence(mel, batch_first=True, padding_value=-20)
            # Segmemt mel-spectrogram into "segment_len" frames.
    if len(mel) > self.segment_len:
      # Randomly get the starting point of the segment.
      start = random.randint(0, len(mel) - self.segment_len)
      # Get a segment with "segment_len" frames.
      mel = torch.FloatTensor(mel[start:start+self.segment_len])
    # compare : when we load training set, we use append to concate datas
    return feat_path, mel

def inference_collate_batch(batch):
  """Collate a batch of data."""
  feat_paths, mels = zip(*batch)

  return feat_paths, torch.stack(mels)

In [ ]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
  """arguments"""
  config = {
    "data_dir": "./Dataset",
    "model_path": "./model.ckpt",
    "output_path": "./output.csv",
  }

  return config


def main(
  data_dir,
  model_path,
  output_path,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  mapping_path = Path(data_dir) / "mapping.json"
  mapping = json.load(mapping_path.open())

  dataset = InferenceDataset(data_dir)

  dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    num_workers=8,
    collate_fn=inference_collate_batch,
  )
  print(f"[Info]: Finish loading data!",flush = True)

  speaker_num = len(mapping["id2speaker"])
  model = Classifier(n_spks=speaker_num).to(device)
  model.load_state_dict(torch.load(model_path))
  model.eval()
  print(f"[Info]: Finish creating model!",flush = True)

  results = [["Id", "Category"]]
  for feat_paths, mels in tqdm(dataloader):
    with torch.no_grad():
#      print(mels.size())
      mels = mels.to(device)
      outs = model(mels)
      preds = outs.argmax(1).cpu().numpy()
      for feat_path, pred in zip(feat_paths, preds):
        results.append([feat_path, mapping["id2speaker"][str(pred)]])

  with open(output_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(results)


if __name__ == "__main__":
  main(**parse_args())

